# How this should work

1. Have a server that is ready to take in audio via REST to 
   1. Transcribe the words
   2. create n-grams of the words
   3. Looks it up if it is contained in wiktionary
   4. creates new anki cards for each, filling with wikt data.
2. On Client: Have GUI that can record audio, then send it to the server. Continuously updates a tsv file with the newly generated vocabulary.

In [1]:
from transformers import pipeline

transcriber = pipeline(
    "automatic-speech-recognition", model="vinai/PhoWhisper-medium", device="cuda"
)

2024-09-21 10:32:10.712906: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-21 10:32:11.530800: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/ducha/mambaforge/envs/ml_gpu/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [2]:
transcriber?

Signature:      transcriber(inputs: Union[numpy.ndarray, bytes, str], **kwargs)
Type:           AutomaticSpeechRecognitionPipeline
String form:    <transformers.pipelines.automatic_speech_recognition.AutomaticSpeechRecognitionPipeline object at 0x766fc850c670>
File:           ~/mambaforge/envs/ml_gpu/lib/python3.10/site-packages/transformers/pipelines/automatic_speech_recognition.py
Docstring:     
Pipeline that aims at extracting spoken text contained within some audio.

The input can be either a raw waveform or a audio file. In case of the audio file, ffmpeg should be installed for
to support multiple audio formats

Example:

```python
>>> from transformers import pipeline

>>> transcriber = pipeline(model="openai/whisper-base")
>>> transcriber("https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/1.flac")
{'text': ' He hoped there would be stew for dinner, turnips and carrots and bruised potatoes and fat mutton pieces to be ladled out in thick, peppered flour-fatten sauce.'

In [3]:
output = transcriber(
    "/home/ducha/Workspace/python/cloze-wikt-cards/asr-adder/notebooks/sample.wav"
)["text"]

/home/ducha/mambaforge/envs/ml_gpu/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Pl

In [4]:
output

'xin chào quý vị và các bạn.'

In [5]:
output = transcriber(
    "/home/ducha/Workspace/python/cloze-wikt-cards/asr-adder/notebooks/sample.wav",
    generate_kwargs={
        "task": "translate",
        "language": "<|vi|>",  # Specify the source language, e.g., French
    },
)["text"]

/home/ducha/mambaforge/envs/ml_gpu/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed task=translate, but also have set `forced_decoder_ids` to [[1, None], [2, 50359]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=translate.


In [6]:
output

'xin chào quý vị và các bạn.'